In [2]:
import altair as alt
import geopandas as gpd
import pandas as pd
from shapely.ops import orient

In [3]:
dc_wards = gpd.read_file('Wards_from_2012_with_Census_2020.geojson')
dc_wards.geometry = dc_wards.geometry.apply(orient, args=(-1,))


In [4]:
basechart = alt.Chart(dc_wards).mark_geoshape(fill='None',stroke='gray').properties(
    width=750*1.75,
    height=500*1.75,
    )

In [5]:
# Define which attributes to lookup from airports.csv
cabi_stations = 'https://raw.githubusercontent.com/mlinds/cabi-data/main/data/stationLookup.csv'
connections_path = 'https://raw.githubusercontent.com/mlinds/cabi-data/main/data/connections_csv.csv'


In [6]:
lookup_data = alt.LookupData(
    cabi_stations, key="short_name", fields=["name", "lat", "lon"]
)

## TODO app popularity to the stations data
lookup_pop = alt.LookupData(
    connections_path, key="st", fields=["popularity"]
)

In [7]:
# Create mouseover selection
select_station = alt.selection_single(
    on="mouseover", nearest=True, fields=["st"], empty="none"
)

In [8]:
color_size_scale = "quantile"
colorscale = alt.Color('popularity:Q', scale=alt.Scale(scheme="inferno",type=color_size_scale))

In [9]:
connections = alt.Chart(connections_path).mark_rule().encode(
    latitude="lat:Q",
    longitude="lon:Q",
    latitude2="lat2:Q",
    longitude2="lon2:Q",
    color=colorscale,
    strokeWidth=alt.StrokeWidth('popularity:Q',scale=alt.Scale(type=color_size_scale)),
    order = alt.Order('popularity:Q',sort="ascending"),
    # opacity = alt.Opacity('popularity:Q',scale=alt.Scale(type="linear"))
).transform_lookup(
    lookup="st",
    from_=lookup_data
).transform_lookup(
    lookup="en",
    from_=lookup_data,
    as_=["name2", "lat2", "lon2"]
).transform_filter(
    select_station
)

annotation = alt.Chart(cabi_stations).mark_text().encode(
    text='name:N'
).transform_lookup(
    lookup = 'short_name',
    from_ = alt.LookupData(cabi_stations, key="short_name", fields=["short_name"]),
    as_ = ["st"]
).transform_filter(
    select_station
)

In [10]:
points = alt.Chart(cabi_stations).mark_circle().encode(
    latitude = 'lat:Q',
    longitude = 'lon:Q',
    tooltip=["name:N"],
).transform_lookup(
    lookup = 'short_name',
    from_ = alt.LookupData(cabi_stations, key="short_name", fields=["short_name"]),
    as_ = ["st"]
).add_selection(
    select_station
)


In [11]:
combined_chart = (connections + points )
combined_chart.configure_view(stroke=None)


alt.LayerChart(...)

In [12]:
html_out = combined_chart.to_html()
with open('MapOutput.html','w') as file:
    file.write(html_out)

with open('vega-lite-map.json','w') as file:
    file.write(combined_chart.to_json())